# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298683350118                   -0.85    5.4    213ms
  2   -8.300234159617       -2.81       -1.26    1.1   91.9ms
  3   -8.300442312294       -3.68       -1.89    2.4    109ms
  4   -8.300460951864       -4.73       -2.75    2.0    118ms
  5   -8.300464134387       -5.50       -3.13    2.1    116ms
  6   -8.300464427934       -6.53       -3.31    1.2   96.4ms
  7   -8.300464535835       -6.97       -3.46    1.0   89.3ms
  8   -8.300464593552       -7.24       -3.60    1.1   90.3ms
  9   -8.300464630509       -7.43       -3.81    1.1   93.2ms
 10   -8.300464638307       -8.11       -3.94    1.6    109ms
 11   -8.300464643222       -8.31       -4.24    1.0   94.8ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      8
│      9
│      7
│      3
│     10
│      4
│      4
│      6
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.66641089903                   -0.71    6.4    607ms
  2   -16.67873244967       -1.91       -1.13    1.1    210ms
  3   -16.67923097237       -3.30       -1.86    3.5    303ms
  4   -16.67927532190       -4.35       -2.71    1.4    218ms
  5   -16.67928552174       -4.99       -3.08    4.0    335ms
  6   -16.67928615921       -6.20       -3.48    3.0    249ms
  7   -16.67928620979       -7.30       -4.04    2.1    242ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32120189289                   -0.56    7.2    1.46s
  2   -33.33282108435       -1.93       -1.00    1.2    679ms
  3   -33.33583966732       -2.52       -1.70    5.0    975ms
  4   -33.33610119725       -3.58       -2.58    4.0    829ms
  5   -33.33680860392       -3.15       -2.25    8.0    1.46s
  6   -33.33684317861       -4.46       -2.32    2.1    696ms
  7   -33.33693910676       -4.02       -3.04    1.1    697ms
  8   -33.33694200104       -5.54       -3.18    1.8    769ms
  9   -33.33694262827       -6.20       -3.28    1.6    966ms
 10   -33.33694351018       -6.05       -3.60    2.5    960ms
 11   -33.33694377604       -6.58       -4.74    2.5    792ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298431520438                   -0.85    5.2    234ms
  2   -8.300242550106       -2.74       -1.59    1.2   85.5ms
  3   -8.300412279984       -3.77       -2.52    2.1    100ms
  4   -8.300366801828   +   -4.34       -2.26    3.4    155ms
  5   -8.300463869429       -4.01       -3.34    1.0   80.0ms
  6   -8.300464534656       -6.18       -3.67    2.2    129ms
  7   -8.300464633225       -7.01       -4.18    1.0   80.2ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32278094135                   -0.56    7.5    1.51s
  2   -33.30938767880   +   -1.87       -1.25    2.2    653ms
  3   -10.10422569369   +    1.37       -0.38    6.6    1.52s
  4   -33.18447193015        1.36       -1.26    5.6    1.34s
  5   -33.25885608840       -1.13       -1.42    3.9    1.01s
  6   -30.28349893574   +    0.47       -0.83    3.8    983ms
  7   -33.33078218018        0.48       -2.02    5.0    1.08s
  8   -33.33656348752       -2.24       -2.36    2.4    775ms
  9   -33.33626020596   +   -3.52       -2.40    2.9    861ms
 10   -33.33626929540       -5.04       -2.58    1.6    677ms
 11   -33.33692378822       -3.18       -2.97    1.8    663ms
 12   -33.33693028713       -5.19 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.